In [13]:
import torch.nn as nn
import torch 

model = nn.Linear(100, 10)

optim = torch.optim.Adam(model.parameters(), lr = 1)
max_epochs = 5000

"""
scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lambda epoch: 1 - epoch/max_epochs, -1)

for _ in range(max_epochs):
    for param_group in optim.param_groups:
        print(param_group['lr'])
    scheduler.step()
"""

"\nscheduler = torch.optim.lr_scheduler.LambdaLR(optim, lambda epoch: 1 - epoch/max_epochs, -1)\n\nfor _ in range(max_epochs):\n    for param_group in optim.param_groups:\n        print(param_group['lr'])\n    scheduler.step()\n"

In [21]:
ctr = 0
def foo():
    global ctr
    ctr += 1
    return ctr

print(foo()
foo()

2

In [14]:
sd = optim.state_dict()
print(sd)

optim_2 = torch.optim.Adam(model.parameters(), lr = .001)
print(optim_2.state_dict())

optim_2.load_state_dict(sd)
print(optim_2.state_dict())


{'state': {}, 'param_groups': [{'lr': 1, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'params': [4401396616, 4401393952]}]}
{'state': {}, 'param_groups': [{'lr': 0.001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'params': [4401396616, 4401393952]}]}
{'state': {}, 'param_groups': [{'lr': 1, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'params': [4401396616, 4401393952]}]}


In [167]:
import torchvision
import torchvision.transforms as transforms


trainset = torchvision.datasets.CIFAR10(
    root=DEFAULT_SN_GAN_DATA_PATH, 
    train=True,
    download=True, 
    # transform=transforms.ToTensor()
)

trainloader = torch.utils.data.DataLoader(
    trainset, 
    batch_size=64,
    num_workers=1,
    shuffle=True
)

Files already downloaded and verified


In [213]:
n = 100
image = trainset[n][0]

arr = torchvision.transforms.ToTensor()(image)

arr = arr*2 - 1
print(arr.shape)

img = torchvision.transforms.ToPILImage()(arr)

img.show()

torch.Size([3, 32, 32])


tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],

In [34]:
class ConditionalBatchNorm2d(nn.Module):
    # Thanks to https://github.com/Kaixhin for this layer
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_features = num_features
        self.bn = nn.BatchNorm2d(num_features, affine=False)
        self.embed = nn.Embedding(num_classes, num_features * 2)
        self.embed.weight.data[:, :num_features] = 1  
        self.embed.weight.data[:, num_features:] = 0  

    def forward(self, x, y):
        out = self.bn(x)
        gamma, beta = self.embed(y).chunk(2, 1)
        out = gamma.view(-1, self.num_features, 1, 1) * out + beta.view(-1, self.num_features, 1, 1)
        return out
    
ConditionalBatchNorm2d(128, 10).embed.weight.data

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]])

In [3]:
from cifar10_models import Cifar10Discriminator
import torch 


dis = Cifar10Discriminator(n_classes=5)

im = torch.rand(94, 3, 32, 32) * 2 - 1
labels = torch.ones((94, ), dtype=torch.long)

dis(im, labels)

p shape:  torch.Size([94, 128])
label weight shape: torch.Size([94, 128])
label_weights * p shape: torch.Size([94, 128])
torch.Size([94, 1])


tensor([[-18.4368],
        [-17.7502],
        [-18.1937],
        [-18.2506],
        [-17.7299],
        [-17.4220],
        [-17.7976],
        [-17.4032],
        [-18.1591],
        [-17.5930],
        [-17.7073],
        [-17.3667],
        [-17.6906],
        [-18.0051],
        [-18.2365],
        [-17.4930],
        [-18.0018],
        [-17.9581],
        [-17.9253],
        [-17.1292],
        [-17.3356],
        [-17.7500],
        [-17.6907],
        [-18.5198],
        [-17.4648],
        [-17.8622],
        [-17.9219],
        [-18.0552],
        [-17.3677],
        [-18.0826],
        [-17.3437],
        [-17.3026],
        [-18.3996],
        [-17.8657],
        [-17.5924],
        [-17.0631],
        [-17.8615],
        [-17.8787],
        [-17.6510],
        [-17.2374],
        [-17.5856],
        [-18.1056],
        [-17.9741],
        [-17.7555],
        [-17.0572],
        [-17.3080],
        [-17.3251],
        [-17.7208],
        [-17.6329],
        [-17.8323],
